In [ ]:
import os
import glob

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xarray as xa
from matplotlib.colors import LogNorm

In [ ]:
buoys = pd.read_feather('data/segments.feather').dropna()
idir = '/Volumes/sat_downloads/NSIDC_iceage'

In [ ]:
for year in range(2023, 1991, -1):
    ofile = f'data/buoys_nsidc_{year}.feather'
    if os.path.exists(ofile):
        print(f'{ofile} exists, skipping')
        continue

    print(f'Processing {year}')
    sia_files = sorted(glob.glob(f'{idir}/iceage_nh_12.5km_{year}*_v4.1.nc'))
    print(len(sia_files))

    with xa.open_dataset(sia_files[0]) as ds:
        ds_x = ds.x.values
        ds_y = ds.y.values
        ds_sia = ds.age_of_sea_ice.values
        ds_time = ds.time.values
    ds_time0 = pd.Timestamp(ds_time[0].strftime('%Y-%m-%d'))
    ds_time1 = pd.Timestamp(ds_time[-1].strftime('%Y-%m-%d'))

    buoys_sub = buoys[(buoys.index.year == year)].copy()
    print(buoys_sub.shape)

    ds_time_array = np.array([np.datetime64(pd.Timestamp(dst.strftime('%Y-%m-%d'))) for dst in ds_time])
    time_indices = np.array([np.argmin(np.abs(ds_time_array - np.datetime64(date))).item() for date in buoys_sub.index])
    
    cols = np.round((buoys_sub.x - ds_x[0]) / (ds_x[-1] - ds_x[0]) * ds_x.size).astype(int)
    rows = np.round((buoys_sub.y - ds_y[0]) / (ds_y[-1] - ds_y[0]) * ds_y.size).astype(int)
    rows = np.clip(rows, 0, ds_y.size - 1)
    cols = np.clip(cols, 0, ds_x.size - 1)
    buoy_sia = ds_sia[time_indices, rows, cols]
    buoys_sub['sia'] = buoy_sia
    buoys_sub.dropna().to_feather(ofile)